In [1]:
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

### KNN 알고리즘, Kmeans
- KNN 지도학습(회귀, 분류), Kmeans 비지도학습

### K-means
- k개를 지정하여 평균을 통해 군집화한다는 뜻으로 생각해보자!
- 비지도학습이라는 게 정답을 준 것이 아니라 정답이 없는 데이터에서 머신에게 패턴을 찾아달라고 하는 것
- y 라벨이 없는 상태에서 데이터를 집어 넣고 특성을 군집화하는 것
- 군집화는 같은 패턴끼리 묶는다는 것
- 작동원리가 어떤 식으로 진행하는지는 '평균'
- 거리 기반으로 작동한다. (유클리디안 거리)
- k-means의 k는 몇 개의 클러스터(군집화)를 할 것인가?
- k는 분석가가 초기값을 입력해야한다. 예를 들어 3개라고 하면 3개의 군집 포인트가 데이터의 패턴을 거리 기반(평균) 군집을 만든다.
- 초기값을 지정해 줘야함

- 단점은 둘 중 거리 기반으로만 하기 때문에 거리가 55:45와 같이 비슷한 거리 차이어도 55에 가깝게 하드클로스터링을 하기 때문에 이 부분이 문제가 된다.
- 거리 기반이라 이상치에 민감해진다.
- 단순히 거리 기반, 평균으로 진행해서 그룹 자체의 분산 구조는 반영하지 못한다.
- 데이터 양이 많아서 반복횟수가 많아지면 속도가 느린 부분도 있다.
- 몇 개의 군집을 선택해야 하는지 가이드를 주는 것이 어렵다.

- 하지만 좋은 점은...
- 군집화하기에 가장 많이 활용된다.
- 이해가 쉽다. 알고리즘이 간결하다. 거리 기반, 평균으로 이해하면 되니까.

In [2]:
df = pd.read_csv("wine_test.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash',
       'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline', 'class'],
      dtype='object')

In [4]:
df = df[['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash',
       'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline', 'class']]
# 컬럼 정리

In [5]:
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0
1,NaN,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,2
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,2
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,2
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,2


In [6]:
df["class"].value_counts().to_frame()

,class
1,71
0,59
2,48


### kmeans 사용하기

In [7]:
kmeans = KMeans(n_clusters=3, init="k-means++", max_iter=300, random_state=111)

In [8]:
kmeans

KMeans(n_clusters=3, random_state=111)

In [9]:
df.isna().sum()

alcohol                         2
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            5
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
class                           0
dtype: int64

In [10]:
df.dropna(inplace=True) # 결측치는 그냥 날림

### kmeans 학습시키는 것은 간단하다.
- fit 안에 넣으면 된다.
- 기존 class 컬럼이 있는데 이 컬럼과 비교해서 kmeans class를 제외한 다른 컬럼으로 군집을 했을 때
- 기존과 비교해서 정말 잘 군집화 하는지 확인해 보자!

In [11]:
df.columns

Index(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline', 'class'],
      dtype='object')

In [12]:
df_sp = df[['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline']]
# class가 없는 와인 데이터

In [13]:
kmeans.fit(df_sp)

KMeans(n_clusters=3, random_state=111)

In [14]:
# 실제로 군집이 어떻게 되었나?
print(kmeans.labels_) # 자동으로 군집화된 내용을 공유해 준다.

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 1 0 0
 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 2 1 2 2 1 2 2 1 1 2 2 0 1 2 2
 2 1 2 2 1 1 2 2 2 2 2 1 1 2 2 2 2 2 1 2 1 2 1 2 1 2 2 2 2 1 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 1 1 1 1 2 2 2 1 1 2 2 1 1 2 1 1 2 2 2 2 1 1
 1 2 1 1 1 2 1 2 1 1 2 1 1 1 1 2 2 1 1 1 1 1 2]


In [15]:
# cluster는 우리가 비지도학습으로 만든 컬럼
df_sp["cluster"] = kmeans.labels_

In [16]:
df_sp["class"] = df["class"]

In [17]:
df_sp

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,cluster,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1,0
5,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,1,2
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,1,2
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,1,2
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,1,2


In [18]:
df_sp.groupby(["class","cluster"])["proline"].count()

class  cluster
0      0          44
       1          13
1      0           1
       1          17
       2          48
2      1          29
       2          19
Name: proline, dtype: int64

- 기존 군집이랑 비교했을 때 조금 다른 결과값이 나왔다.
- kmeans는 비지도학습이라 정답이 없다.
- 예시로 기존 class와 비교를 한 것이지, kmeans가 작동하는 원리는 평균 기반 거리이기 때문에 군집이 다를 수 있다.
- 기존의 class가 어떤 식으로 되어있는지는 우리가 모르니깐...

In [19]:
df_sp.groupby(["cluster"])["proline"].count()

cluster
0    45
1    59
2    67
Name: proline, dtype: int64

In [20]:
df_sp.groupby(["class"])["proline"].count()

class
0    57
1    66
2    48
Name: proline, dtype: int64

- 거리 기반으로 군집화를 진행한다고 했으니...
- 변수에 대해서 한번 조정해보고 새롭게 kmeans를 진행해보자!

- 해당 컬럼이 차원이 10개 넘는 고차원인 상태
- 해당 고차원의 상태의 데이터에서 거리 기반으로 군집을 만드는 것
- 무작정 Kmeans를 모든 컬럼을 다 이용해서 진행하면 '차원의 저주'라고 하는 문제가 발생할 수 있다.
- 컬럼이 많아지면 고차원에서는 거리가 미세하게만 바뀌고 거의 바뀌지 않을 수도 있고, 그렇게 되면 군집화가 잘 될 수 있을지 미지수...
- 유의미한 변수가 무엇인지?
- 어떤 변수를 사용해야 하는가?
- 의미 있는 변수로만 최대한 사용해야한다.

- 의미있는 변수가 무엇인가?
- 분석가가 직접 컬럼을 선별할 수 있다.
- 기술적인 것은 피처에 대한 중요성을 보고 컬럼을 추출할 수 있고...
- 중요한 부분) 모든 컬럼을 다 kmeans에 사용하는 것은 잘못된 방향이다.

In [21]:
df_sp

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,cluster,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1,0
5,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,1,2
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,1,2
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,1,2
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,1,2


- 위의 컬럼 중 class별로 평균을 비교해보자!
- 평균의 차이가 많이 나는 경우는 어느정도 이상치가 있을 것이라 판단하고 해당 컬럼은 제외하고 평균이 비슷한 컬럼들만 선택해보자!

In [22]:
# 기존 class 기반으로 볼 때 각 컬럼들의 평균의 차이를 보고 컬럼을 선별하여 다시 군집화 해보자!
df # 초기 데이터셋에서 class를 제거한 거 (결측치도 제거) 다시 갖고오기

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,0
5,14.20,1.76,2.45,15.2,112,3.27,3.39,0.34,1.97,6.75,1.05,2.85,1450,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,2
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,2
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,2
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,2


In [23]:
a = list(df.columns)

In [24]:
a

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline',
 'class']

In [25]:
for i in a:
    print("--------", i, "mean 비교")
    print(df.groupby(["class"])[i].mean())

-------- alcohol mean 비교
class
0    13.735263
1    12.294242
2    13.153750
Name: alcohol, dtype: float64
-------- malic_acid mean 비교
class
0    2.021228
1    1.932121
2    3.333750
Name: malic_acid, dtype: float64
-------- ash mean 비교
class
0    2.466140
1    2.245606
2    2.437083
Name: ash, dtype: float64
-------- alcalinity_of_ash mean 비교
class
0    17.192982
1    20.257576
2    21.416667
Name: alcalinity_of_ash, dtype: float64
-------- magnesium mean 비교
class
0    106.614035
1     93.454545
2     99.312500
Name: magnesium, dtype: float64
-------- total_phenols mean 비교
class
0    2.844211
1    2.254848
2    1.678750
Name: total_phenols, dtype: float64
-------- flavanoids mean 비교
class
0    2.986316
1    2.089545
2    0.781458
Name: flavanoids, dtype: float64
-------- nonflavanoid_phenols mean 비교
class
0    0.290526
1    0.365152
2    0.447500
Name: nonflavanoid_phenols, dtype: float64
-------- proanthocyanins mean 비교
class
0    1.908772
1    1.612879
2    1.153542
Name: proanthocya

### 해당 컬럼이 차이가 많이 나지 않는다 판단해서 선택
alcohol ash nonflavanoid_phenols proanthocyanins

In [27]:
df_sp_1 = df[["alcohol", "ash", "nonflavanoid_phenols", "proanthocyanins"]]

In [28]:
df_sp_1

,alcohol,ash,nonflavanoid_phenols,proanthocyanins
0,14.23,2.43,0.28,2.29
2,13.16,2.67,0.30,2.81
3,14.37,2.50,0.24,2.18
4,13.24,2.87,0.39,1.82
5,14.20,2.45,0.34,1.97
...,...,...,...,...
173,13.71,2.45,0.52,1.06
174,13.40,2.48,0.43,1.41
175,13.27,2.26,0.43,1.35
176,13.17,2.37,0.53,1.46


In [29]:
kmeans.fit(df_sp_1)

KMeans(n_clusters=3, random_state=111)

In [30]:
print(kmeans.labels_)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 2 1 1 1 1 1 1 1 1 2 2 2
 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 0 0 0 1 2 2 0 1 2 1 0 0 2
 0 0 0 0 0 0 2 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 0 2 0 0 0 2 0 0 0 2 0 0 0 0 0
 0 2 0 0 0 1 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 1 2
 2 1 2 1 1 2 2 2 2 2 2 1 2 1 2 2 2 1 2 2 2 2 1]


In [31]:
df_sp_1["cluster"] = kmeans.labels_

C:\Users\doram\AppData\Local\Temp\ipykernel_18536\773804852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp_1["cluster"] = kmeans.labels_


In [32]:
df_sp_1.groupby(["cluster"])["ash"].count()

cluster
0    48
1    61
2    62
Name: ash, dtype: int64

- 이걸 통해서 어떤 인사이트를 얻는 것인가?
- 군집화를 통해서 무엇이 중요한 것이고 무엇을 얻는가?
- 현업에서도 마찬가지고...

- 군집이 형성된다는 것은 해당 0, 1, 2에 따른 데이터의 패턴이 다를 것이다는 것.
- 1 군집에 대한 다른 컬럼들의 값의 분포
- 비지도학습이기 때문에 군집이 없는 상태였는데
- 예시를 위해 비교 class가 있었으나
- 없다는 가정으로 한다면 흩어진 데이터의 분포에서 군집을 만들어 주었으니
- 우리는 해당 군집에 대한 다른 컬럼들의 데이터 특성을 보고 그것을 이해하고 실무에 적용시키거나 사용하거나 할 수 있다.

- ex) 고객군의 데이터라고 한다면
- 고객군 군집이 형성되었는데
- 컬럼이 구매주기, 구매시간, 구매빈도 등의 컬럼이라면
- 새롭게 클러스터된 해당 그룹과 기존 컬럼들의 값을 비교하면서 새로운 인사이트를 찾을 수 있을 것

In [33]:
df_sp_1

,alcohol,ash,nonflavanoid_phenols,proanthocyanins,cluster
0,14.23,2.43,0.28,2.29,1
2,13.16,2.67,0.30,2.81,1
3,14.37,2.50,0.24,2.18,1
4,13.24,2.87,0.39,1.82,1
5,14.20,2.45,0.34,1.97,1
...,...,...,...,...,...
173,13.71,2.45,0.52,1.06,2
174,13.40,2.48,0.43,1.41,2
175,13.27,2.26,0.43,1.35,2
176,13.17,2.37,0.53,1.46,2


### 거리 기반으로 이야기했으면...
- 거리 기반이기 때문에 값에 대한 차이에 민감할 것
- 클러스터링할 때 스케일링을 해야 한다.
- 스케일링을 해보자!

In [34]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [35]:
st_scaler = StandardScaler()

In [36]:
st_scaler.fit(df_sp_1)

StandardScaler()

In [37]:
df_sp_1_sc = st_scaler.transform(df_sp_1)

In [39]:
df_tt_sc = pd.DataFrame(df_sp_1_sc, columns=df_sp_1.columns)

In [40]:
df_tt_sc

,alcohol,ash,nonflavanoid_phenols,proanthocyanins,cluster
0,1.526314,0.207821,-0.661032,1.250922,-0.102614
1,0.181214,1.080798,-0.502496,2.170422,-0.102614
2,1.702309,0.462439,-0.978105,1.056412,-0.102614
3,0.281782,1.808278,0.210918,0.419835,-0.102614
4,1.488601,0.280569,-0.185423,0.685075,-0.102614
...,...,...,...,...,...
166,0.872621,0.280569,1.241406,-0.924050,1.150747
167,0.482919,0.389691,0.527991,-0.305156,1.150747
168,0.319495,-0.410537,0.527991,-0.411252,1.150747
169,0.193785,-0.010423,1.320674,-0.216742,1.150747


In [41]:
kmeans.fit(df_tt_sc)

KMeans(n_clusters=3, random_state=111)

In [42]:
print(kmeans.labels_)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0
 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 2 2 2 2 0 0 2 1 0 1 2 2 0
 2 2 2 2 2 2 0 2 0 2 2 2 2 2 2 0 2 2 2 2 2 2 2 0 2 2 2 0 2 2 2 0 2 2 2 2 2
 2 0 2 2 2 1 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 1 0 1 0 0
 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0 1]


In [43]:
df_tt_sc["cluster"] = kmeans.labels_

In [44]:
df_tt_sc.groupby(["cluster"])["ash"].count()

cluster
0    58
1    64
2    49
Name: ash, dtype: int64

- 거기 기반으로 민감하게 움직이기 때문에 스케일링과 변수들의 이상치 등을 고려하여 군집을 만들어야 합니다.

- 몇 개의 군집이 정말 적정한 것인가?
- 위 질문에 대한 답변으로 바라보는 점수들이 있다.
- inertia, 실루엣 점수, 엘보우 점수
- 이러한 점수를 통해서 군집의 점수를 판단하는 것인데...
- 점수가 잘 나온다고해서 해당 군집의 개수를 선정하는 것이 아니라
- 정말 군집이 잘 형성되었는지 시각화를 통해서도 봐야한다.

In [45]:
df_tt_sc

,alcohol,ash,nonflavanoid_phenols,proanthocyanins,cluster
0,1.526314,0.207821,-0.661032,1.250922,1
1,0.181214,1.080798,-0.502496,2.170422,1
2,1.702309,0.462439,-0.978105,1.056412,1
3,0.281782,1.808278,0.210918,0.419835,1
4,1.488601,0.280569,-0.185423,0.685075,1
...,...,...,...,...,...
166,0.872621,0.280569,1.241406,-0.924050,0
167,0.482919,0.389691,0.527991,-0.305156,0
168,0.319495,-0.410537,0.527991,-0.411252,0
169,0.193785,-0.010423,1.320674,-0.216742,0


- 4차원 이상의 차원을 시각화하게 되면 우리는 이 부분을 이해할 수 없다.
- 이런 다차원의 특성을 확인하기 위해서 차원 축소를 하는 것이다. pca
- 차원 축소를 하는 이유가 고차원에 대한 변수들을 군집화했을 때 시각화하면 볼 수가 없다.
- 이것을 2개의 차원으로 축소하여 군집들을 시각화해보면서 정말 잘 시각화되었는지 확인하는 것

### 필수과제 1
- from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
- 스케일링 진행하지 않은 2개 + 추가로 더 하셔도 됩니다 abs, 등으로 차이를 비교해주세요!

### 필수과제 2
- 고객데이터를 올릴 것
- 해당 고객데이터를 기반으로 여러분들이 가설을 세워서 어떤 컬럼을 사용할 것인지 정리하고
- 해당 컬럼을 기반으로 kmeans를 진행하여서 분석에 대한 인사이트를 정리해 주세요!
- 군집에 대해서 몇 개를 선정하는지에 대한 질문의 답변은 위에 적어놨으니
- 가능하신 분은 구글링을 통해서 해당 지표도 만들어 보시면 좋을 것 같습니다.
- 중요한 것은 인사이트
- 기존에 데이터가 어떤 패턴을 보였고, 우리가 선정한 컬럼을 통해서 만들어진 군집(고객군)이 어떤 패턴을 보이는지?